# Notebook exploration données : job_events

In [ ]:
import os, json
import pandas as pd
from azure_blob import download_blob_file
from utilities import string_to_dict
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
filename = 'metrics.csv'
path = '../data/'

In [ ]:
# téléchargement dans le repertoire 'data' d'un fichiers 'csv' depuis le blob
download_blob_file(file_name=filename, local_path=path)
metrics = os.path.join(path, filename)

In [ ]:
# création d'un dataframe à partir du csv de données
metrics_df = pd.read_csv(metrics).sort_values(by='created_at')
metrics_df.info()

In [ ]:
metrics_df.dropna(axis=1, inplace=True)
metrics_df.drop('machineId', axis=1, inplace=True)
# metrics_df.set_index('id')

In [ ]:
metrics_df.info()

In [ ]:
def convert_str_to_list(series):
    return series.apply(lambda x :json.loads(x))

In [ ]:
def get_keys(dict):
    return list(k for k in dict.keys() )

In [ ]:
def add_key_column(df, dict_col_name, list_keys):
    for k in list_keys:
        df[k] = df.apply(lambda x: x[dict_col_name].get(k), axis=1)
    df.drop(dict_col_name, axis=1, inplace=True)
    return df

In [ ]:
def check_list_length(list):
    if len(list) :
        print(len(list))

### connected_operators

In [ ]:
metrics_df.connected_operators[0][0].keys()

In [ ]:
#metrics_df.connected_operators = metrics_df.connected_operators.apply(lambda x :json.loads(x)[0])
metrics_df.connected_operators = convert_str_to_list(metrics_df.connected_operators)
list_keys = get_keys(metrics_df.connected_operators)
metrics_df = add_key_column(metrics_df, 'connected_operators', list_keys)

In [ ]:
metrics_df.head(3)

### modules

In [ ]:
metrics_df.modules = convert_str_to_list(metrics_df.modules)
metrics_df['Length'] = metrics_df.modules.map(len)
metrics_df.Length.max(numeric_only=True)

In [ ]:
metrics_df.id.iloc[metrics_df.Length.max(numeric_only=True)]

In [ ]:
metrics_df.loc[metrics_df.id == 10,:]

In [ ]:
list_keys = get_keys(metrics_df.modules[0][0])
print(list_keys)

In [ ]:
for i in range(0, len(metrics_df.modules[0])) :
    for k in list_keys:
        metrics_df[k+'_'+str(i)] = metrics_df.apply(lambda x: x['modules'][0].get(k), axis=1)

In [ ]:
metrics_df

In [ ]:
modules_df = metrics_df['modules'].apply(pd.Series, dtype='object')

### events column

In [ ]:
# convertion de "events" en type list
metrics_df.events = metrics_df.events.apply(lambda x :json.loads(x))

In [ ]:
# eclatement des listes des events dans un dataframe
events_df = pd.DataFrame(metrics_df.events.to_list(), dtype=object)

In [ ]:
# renommage des colonnes du dataframe des events
i = 0
col_names = {}
for col in range(events_df.columns.start, events_df.columns.stop):
    col_names[col] = 'e'+str(col)
events_df.rename(columns=col_names, inplace=True)

In [ ]:
events_df.fillna({}, inplace=True)

In [ ]:
for col in events_df.columns :
    for event in events_df[col].values :
        if not isinstance(event, dict) :
            event = {}

In [ ]:
# on liste les index des lignes qui ont 1 et plus de 1 event
single_event_indexes = []
multiple_events_indexes = []
for index, value in metrics_df.events.items() :
    if len(value) == 1 :
        single_event_indexes.append(index)
    if len(value) > 1 :
        multiple_events_indexes.append(index)

In [ ]:
metrics = metrics_df['events'].apply(pd.Series, dtype='object')

In [ ]:
metrics_events_df = metrics_df[metrics_df['events'].map(lambda d: len(d)) > 0]

In [ ]:
events_df = pd.DataFrame(metrics_events_df.events.to_list())

In [ ]:
events_df = pd.DataFrame(metrics_events_df)
events_df[['source','message','timestamp','criticality','identification']] = pd.DataFrame(events_df.events.tolist(), index= events_df.index)

In [ ]:
# e = []
# for m in metrics_df.events.values :
#     if len(m) > 0 :
#         e.append(m)
# len(e)

In [ ]:
#metrics = metrics_df['events'].apply(pd.Series, dtype='object')

In [ ]:
metrics_df

In [ ]:
metrics_df.columns

In [ ]:
df = metrics_df.loc[:, ['created_at', 'varnishLevelsTargetvolume']]
# Set the Date as Index
df.created_at = pd.to_datetime(df.created_at)
df.index = df['created_at']
del df['created_at']
df.plot(figsize=(15, 6))
plt.show()